In [1]:
import torch
from torch import nn

# 定义前馈网络类
class PositionWiseFFN(nn.Module):
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs):
        super(PositionWiseFFN, self).__init__()
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)
    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

# 初始化前馈网络（设置输出维度不同于输入）
ffn = PositionWiseFFN(4, 8, 4)
ffn.eval()

# 构造一个包含4个位置的句子，每个词是4维（不同值便于对比）
X = torch.tensor([[
    [1.0, 0.0, 0.0, 0.0],  # token 1
    [0.0, 1.0, 0.0, 0.0],  # token 2
    [0.0, 0.0, 1.0, 0.0],  # token 3
    [0.0, 0.0, 0.0, 1.0],  # token 4
]])

# 前向传播
Y = ffn(X)

# 打印前后对比
print("输入 X: (每个位置一个向量)")
for i, vec in enumerate(X[0]):
    print(f"位置 {i}: {vec.numpy()}")

print("\n输出 Y: (每个位置的向量独立变化)")
for i, vec in enumerate(Y[0]):
    print(f"位置 {i}: {vec.detach().numpy()}")

输入 X: (每个位置一个向量)
位置 0: [1. 0. 0. 0.]
位置 1: [0. 1. 0. 0.]
位置 2: [0. 0. 1. 0.]
位置 3: [0. 0. 0. 1.]

输出 Y: (每个位置的向量独立变化)
位置 0: [ 0.2974003   0.11878185  0.03308525 -0.23827533]
位置 1: [ 0.2006384  -0.06407945 -0.12253505 -0.5528127 ]
位置 2: [ 4.6741062e-01 -4.7645345e-04  4.2047398e-04 -2.0997192e-01]
位置 3: [ 1.5580866e-01  2.5901943e-04 -4.5705687e-02 -2.6426166e-01]


针对上面的输出，你会发现：

输入的每个向量是“独立送入”同一个 MLP

在这个上下文中，“每个向量”指的就是 输出张量中每一行，也就是 序列中每一个位置的词向量。

输出形状和顺序保持一致，但数值发生了非线性变化

不同输入位置 → 不同输出，但结构无交叉（互不干扰）
